Rahmanda Afebrio Yuris Soesatyo - Chapter 12:,Custom Models and Training with TensorFlow

1. A Quick Tour of TensorFlow

TensorFlow merupakan library komputasi numerik yang memiliki kemiripan dengan NumPy, namun dirancang khusus untuk kebutuhan pembelajaran mesin dan deep learning berskala besar. TensorFlow dilengkapi berbagai fitur lanjutan, seperti dukungan komputasi pada GPU dan TPU, eksekusi terdistribusi, automatic differentiation (autodiff), serta Just-In-Time (JIT) compiler berbasis computation graph yang memungkinkan eksekusi program menjadi lebih efisien.

Di atas core engine tersebut, TensorFlow menyediakan beragam modul tingkat tinggi yang mendukung seluruh siklus pengembangan model, antara lain:

tf.keras untuk pengembangan model deep learning tingkat tinggi,

tf.data untuk membangun pipeline input data yang efisien dan skalabel,

tf.image dan tf.signal untuk pemrosesan data khusus,

tf.summary untuk visualisasi proses pelatihan menggunakan TensorBoard,

TFX (TensorFlow Extended) untuk membangun production ML pipelines,

TensorFlow Hub sebagai repositori model pra-latih,

serta TensorFlow Lite dan TensorFlow.js untuk deployment pada perangkat mobile dan aplikasi berbasis web.

TensorFlow Architecture

Arsitektur TensorFlow memisahkan antara:

kode Python sebagai high-level API (seperti Keras dan Data API), dan

execution engine berbasis C++ yang bertugas menjalankan komputasi pada CPU, GPU, atau TPU.

Pemisahan ini memungkinkan model untuk dikembangkan dan dilatih di satu lingkungan, kemudian dieksekusi secara efisien di lingkungan lain tanpa memerlukan perubahan besar pada kode program.

2. Custom Losses, Metrics, Initializers, Regularizers, dan Constraints

Bab ini membahas cara mendefinisikan berbagai komponen kustom dalam Keras untuk menyesuaikan kebutuhan model, termasuk loss functions, metrics, activation functions, initializers, regularizers, serta constraints.

Komponen-komponen tersebut umumnya ditulis menggunakan operasi TensorFlow, sehingga dapat dikonversi menjadi computation graph. Dengan demikian, komponen kustom tetap memperoleh manfaat optimisasi TensorFlow dan kompatibilitas penuh dengan ekosistem yang tersedia.

Custom Losses and Metrics

Salah satu contoh penting adalah implementasi Huber loss secara kustom. Fungsi loss ini dapat dibuat:

sebagai fungsi sederhana, atau

sebagai subclass dari keras.losses.Loss, sehingga parameter seperti nilai threshold dapat disimpan secara otomatis saat model disimpan.

Pendekatan yang serupa juga dapat diterapkan pada metrics dengan membuat subclass dari keras.metrics.Metric. Cara ini memungkinkan:

akumulasi state antar-batch selama proses evaluasi,

penyimpanan informasi statistik seperti jumlah true positives dan false positives secara bertahap.

Pendekatan ini sangat berguna untuk metrik evaluasi yang kompleks, terutama ketika perhitungannya tidak dapat dilakukan hanya dalam satu batch data.

In [20]:
import tensorflow as tf
from tensorflow import keras

# Functional form (simple)
def huber_fn(y_true, y_pred, threshold=1.0):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2.0
    linear_loss  = threshold * tf.abs(error) - threshold**2 / 2.0
    return tf.where(is_small_error, squared_loss, linear_loss)

model.compile(loss=lambda y_true, y_pred: huber_fn(y_true, y_pred, 1.0),
              optimizer="nadam")

# Class form (threshold tersimpan di config)
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2.0
        linear_loss  = (self.threshold * tf.abs(error)
                        - self.threshold**2 / 2.0)
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base = super().get_config()
        return {**base, "threshold": self.threshold}

model.compile(loss=HuberLoss(2.0), optimizer="nadam")


In [21]:
import tensorflow as tf
from tensorflow import keras

# Functional form (simple)
def huber_fn(y_true, y_pred, threshold=1.0):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2.0
    linear_loss  = threshold * tf.abs(error) - threshold**2 / 2.0
    return tf.where(is_small_error, squared_loss, linear_loss)

model.compile(loss=lambda y_true, y_pred: huber_fn(y_true, y_pred, 1.0),
              optimizer="nadam")

# Class form (threshold tersimpan di config)
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2.0
        linear_loss  = (self.threshold * tf.abs(error)
                        - self.threshold**2 / 2.0)
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base = super().get_config()
        return {**base, "threshold": self.threshold}

model.compile(loss=HuberLoss(2.0), optimizer="nadam")


3. Custom Layers
Jika dibutuhkan perilaku layer yang tidak tersedia secara langsung di Keras, terdapat beberapa pendekatan yang dapat digunakan:

Lambda layer
Digunakan untuk operasi sederhana tanpa bobot, misalnya fungsi exp atau transformasi element-wise lainnya.

Subclass keras.layers.Layer
Digunakan untuk membuat layer dengan bobot atau perilaku yang lebih kompleks, mirip dengan implementasi layer Dense kustom.

Implementing a Custom Layer
Subclass dari keras.layers.Layer umumnya mengimplementasikan metode berikut:

__init__() untuk menyimpan hyperparameter
build() untuk membuat bobot layer menggunakan add_weight()
call() untuk mendefinisikan proses forward pass
get_config() agar layer dapat di-save dan di-load dengan benar
Untuk layer yang berperilaku berbeda antara training dan inference (seperti Dropout, Batch Normalization, atau Noise layers), metode call() menerima argumen training dan menyesuaikan perilakunya berdasarkan mode tersebut.

In [22]:
from tensorflow import keras
import tensorflow as tf

class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel",
            shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal"
        )
        self.bias = self.add_weight(
            name="bias",
            shape=[self.units],
            initializer="zeros"
        )
        super().build(batch_input_shape)

    def call(self, X):
        z = X @ self.kernel + self.bias
        return self.activation(z) if self.activation is not None else z

    def get_config(self):
        base = super().get_config()
        return {
            **base,
            "units": self.units,
            "activation": keras.activations.serialize(self.activation),
        }

# Use it like a normal layer
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=(10,)),
    MyDense(30, activation="relu"),
    MyDense(1)
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [23]:
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        return X

    def get_config(self):
        base = super().get_config()
        return {**base, "stddev": self.stddev}


4. Custom Models & Losses Based on Internals

Untuk arsitektur jaringan yang lebih kompleks—misalnya model dengan residual blocks, auxiliary heads, atau alur komputasi yang tidak bersifat linear—Keras menyediakan fleksibilitas dengan cara melakukan subclassing terhadap keras.Model.

Dalam pendekatan ini:

Method __init__() digunakan untuk mendefinisikan seluruh layer atau blok penyusun model.

Method call() bertugas mengatur proses forward pass, termasuk logika seperti perulangan, skip connections, percabangan alur, serta output ganda (multi-output).

Pendekatan subclassing memberikan kontrol penuh terhadap struktur dan aliran data di dalam model, sehingga sangat cocok untuk arsitektur non-standar yang sulit direpresentasikan menggunakan Sequential atau Functional API.

Internal-State-Based Losses

Selain loss yang berasal langsung dari perbandingan prediksi dan label, Keras juga mendukung penambahan loss tambahan yang bergantung pada kondisi internal model, seperti nilai aktivasi atau bobot tertentu. Loss semacam ini dapat dimasukkan ke total fungsi objektif menggunakan metode add_loss().

Pendekatan ini bermanfaat untuk:

regularisasi berbasis aktivasi,

penerapan structural constraints pada model,

atau tujuan optimisasi tambahan yang tidak secara eksplisit bergantung pada label target.

Dengan mekanisme ini, proses pelatihan dapat diarahkan tidak hanya untuk meminimalkan kesalahan prediksi, tetapi juga untuk memenuhi karakteristik internal tertentu yang diinginkan dari model.

In [25]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [
            keras.layers.Dense(
                n_neurons, activation="elu", kernel_initializer="he_normal"
            )
            for _ in range(n_layers)
        ]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z  # skip connection


class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(
            30, activation="elu", kernel_initializer="he_normal"
        )
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)


In [26]:

class ReconstructingRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [
            keras.layers.Dense(
                30, activation="selu", kernel_initializer="lecun_normal"
            )
            for _ in range(5)
        ]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)  # auxiliary regularization loss
        return self.out(Z)


5. Automatic Differentiation dengan tf.GradientTape & Custom Gradients

TensorFlow mendukung reverse-mode automatic differentiation melalui mekanisme tf.GradientTape, yang memungkinkan perhitungan gradien secara efisien untuk fungsi yang kompleks terhadap banyak parameter sekaligus.

Di dalam blok with tf.GradientTape():, TensorFlow secara otomatis merekam seluruh operasi yang melibatkan tf.Variable. Setelah proses forward computation selesai, gradien dari suatu nilai skalar—biasanya loss function—dapat dihitung terhadap satu atau lebih variabel menggunakan metode tape.gradient().

Pendekatan ini menjadi fondasi utama dalam proses pelatihan model berbasis gradient-based optimization di TensorFlow.

Penggunaan Lanjutan GradientTape

tf.GradientTape menyediakan sejumlah fitur lanjutan yang meningkatkan fleksibilitas eksperimen, antara lain:

Persistent Tape
Dengan mengaktifkan opsi persistent=True, satu tape yang sama dapat digunakan untuk menghitung gradien lebih dari satu kali. Hal ini berguna pada skenario optimisasi kompleks yang membutuhkan beberapa perhitungan gradien dari ekspresi yang sama.

Watching Tensors
Selain tf.Variable, tape juga dapat secara eksplisit memantau (watch) tensor biasa. Fitur ini diperlukan ketika gradien terhadap input dibutuhkan, misalnya pada regularisasi berbasis sensitivitas atau analisis pengaruh input terhadap output model.

Custom Gradients
Untuk fungsi yang sulit didiferensiasikan secara numerik atau berpotensi tidak stabil, TensorFlow menyediakan dekorator @tf.custom_gradient. Dengan mekanisme ini, rumus gradien dapat didefinisikan secara manual, sehingga proses pelatihan menjadi lebih stabil dan terkontrol.

Kemampuan ini memberikan keleluasaan tinggi dalam merancang metode optimisasi dan eksperimen pelatihan tingkat lanjut yang melampaui konfigurasi standar Keras.

In [27]:

import tensorflow as tf

def f(w1, w2):
    return 3 * w1**2 + 2 * w1 * w2

w1 = tf.Variable(5.0)
w2 = tf.Variable(3.0)

with tf.GradientTape() as tape:
    z = f(w1, w2)

grads = tape.gradient(z, [w1, w2])
print(grads[0].numpy(), grads[1].numpy())  # 36.0, 10.0



36.0 10.0


In [28]:
@tf.custom_gradient
def my_better_softplus(z):
    exp = tf.exp(z)
    y = tf.math.log(exp + 1.0)

    def grad(dy):
        return dy / (1.0 + 1.0 / exp)  # stable derivative

    return y, grad

6. Custom Training Loops & TensorFlow Functions

Walaupun model.fit() di Keras sudah mencakup sebagian besar kebutuhan pelatihan model (sekitar 95% skenario umum), ada kondisi tertentu di mana custom training loop menjadi pilihan yang lebih tepat. Biasanya ini dibutuhkan ketika:

menggunakan lebih dari satu optimizer untuk bagian jaringan yang berbeda,

menerapkan aturan update gradien yang tidak standar,

atau membutuhkan kontrol penuh terhadap logging, monitoring, dan alur training.

Menulis Custom Training Loop

Dengan memanfaatkan tf.GradientTape, proses pelatihan manual umumnya dilakukan melalui tahapan berikut:

Mengambil satu batch data dari dataset

Melakukan forward pass untuk menghasilkan prediksi

Menghitung total loss, termasuk komponen regularisasi dari model.losses

Menghitung gradien loss terhadap parameter model

Menerapkan gradien menggunakan optimizer yang dipilih

Memperbarui nilai metrik secara eksplisit

Pendekatan ini memberikan fleksibilitas maksimal, karena setiap langkah pelatihan dapat dimodifikasi sesuai kebutuhan eksperimen atau desain algoritma tertentu.

TensorFlow Functions (tf.function)

Dekorator tf.function berfungsi mengonversi fungsi Python biasa menjadi TensorFlow Function yang dieksekusi sebagai optimized computation graph. Proses ini dilakukan melalui mekanisme AutoGraph dan graph tracing, sehingga eksekusi menjadi lebih cepat dan efisien dibandingkan eksekusi Python murni.

Agar tf.function bekerja optimal, terdapat beberapa praktik penting yang perlu diperhatikan:

Gunakan operasi TensorFlow (tf.*), bukan NumPy murni

Hindari side-effect Python yang memengaruhi alur logika program

Definisikan variabel di luar fungsi yang didekorasi tf.function

Gunakan tf.range untuk perulangan yang ingin dimasukkan ke dalam computation graph

Dengan mengikuti aturan-aturan tersebut, TensorFlow dapat membangun graph yang stabil, efisien, dan cocok untuk eksekusi skala besar maupun deployment.

In [ ]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
import keras

# Model
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

# Random mini-batch sampler
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

# Status bar
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(
        "{}: {:.4f}".format(m.name, m.result())
        for m in [loss] + (metrics or [])
    )
    end = "" if iteration < total else "\n"
    print("\r{}/{} - {}".format(iteration, total, metrics), end=end)

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error

mean_loss = keras.metrics.Mean(name="loss")
metrics = [keras.metrics.MeanAbsoluteError(name="mae")]

for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)  # include reg losses
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        mean_loss.update_state(loss)
        for metric in metrics:
            metric.update_state(y_batch, y_pred)

        print_status_bar(step * batch_size, len(X_train), mean_loss, metrics)

    print_status_bar(len(X_train), len(X_train), mean_loss, metrics)
    for m in [mean_loss] + metrics:
        m.reset_states()


In [31]:
@tf.function
def train_step(X_batch, y_batch):
    with tf.GradientTape() as tape:
        y_pred = model(X_batch, training=True)
        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    mean_loss.update_state(loss)
    for metric in metrics:
        metric.update_state(y_batch, y_pred)
